In [1]:
import os, sys, json
from dotenv import load_dotenv
import openai

In [2]:
# 1) load key from .env
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")
if not openai.api_key:
    print("⚠️  Missing OPENAI_API_KEY in .env", file=sys.stderr)
    sys.exit(1)


In [3]:
# 2) list available models
print("Models you can use:")
for m in openai.models.list().data:
    print("  -", m.id)
print()

Models you can use:
  - gpt-4-0613
  - gpt-4
  - gpt-3.5-turbo
  - gpt-4o-audio-preview-2025-06-03
  - gpt-4.1-nano-2025-04-14
  - gpt-4.1-nano
  - gpt-image-1
  - gpt-4o-realtime-preview-2025-06-03
  - davinci-002
  - babbage-002
  - gpt-3.5-turbo-instruct
  - gpt-3.5-turbo-instruct-0914
  - dall-e-3
  - dall-e-2
  - gpt-4-1106-preview
  - gpt-3.5-turbo-1106
  - tts-1-hd
  - tts-1-1106
  - tts-1-hd-1106
  - text-embedding-3-small
  - text-embedding-3-large
  - gpt-4-0125-preview
  - gpt-4-turbo-preview
  - gpt-3.5-turbo-0125
  - gpt-4-turbo
  - gpt-4-turbo-2024-04-09
  - gpt-4o
  - gpt-4o-2024-05-13
  - gpt-4o-mini-2024-07-18
  - gpt-4o-mini
  - gpt-4o-2024-08-06
  - chatgpt-4o-latest
  - o1-preview-2024-09-12
  - o1-preview
  - o1-mini-2024-09-12
  - o1-mini
  - gpt-4o-realtime-preview-2024-10-01
  - gpt-4o-audio-preview-2024-10-01
  - gpt-4o-audio-preview
  - gpt-4o-realtime-preview
  - omni-moderation-latest
  - omni-moderation-2024-09-26
  - gpt-4o-realtime-preview-2024-12-17
  - 

In [4]:
# 3) Simple ChatCompletion
response = openai.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user",   "content": "Give me three bullet points on why AI is exciting."}
    ],
    temperature=0.7,
    max_tokens=100
)

In [5]:
print("Chat response:")
print(response.choices[0].message.content.strip())
print()

Chat response:
1. AI has the potential to revolutionize various industries by increasing efficiency and productivity through automation and advanced data analysis.
2. AI technology is constantly evolving and improving, leading to exciting breakthroughs in areas such as self-driving cars, natural language processing, and medical diagnostics.
3. AI has the ability to tackle complex problems and make predictions that were previously impossible with traditional methods, opening up new possibilities for innovation and discovery.



In [14]:
# 4) Streaming response example (updated)
print("Streaming response:")
stream = openai.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are an assistant that streams your reply."},
        {"role": "user",   "content": "Stream me a haiku about coding."}
    ],
    temperature=0.5,
    stream=True,
)
# Each `chunk` is a ChatCompletionChunk; delta.content holds the new text (or None)
for chunk in stream:
    delta_text = chunk.choices[0].delta.content
    if delta_text:
        print(delta_text, end="", flush=True)
print("\n")

Streaming response:
In lines of code flow  
Logic and creativity  
Programmers create



In [9]:
# 5) Function-calling stub
def get_current_weather(location: str) -> dict:
    # Dummy implementation — replace with real API calls if desired
    return {"location": location, "weather": "sunny", "temperature": "25°C"}

response = openai.chat.completions.create(
    model="gpt-3.5-turbo-1106",  # or simply "gpt-3.5-turbo"
    messages=[
        {"role": "system", "content": "You are a weather bot."},
        {"role": "user",   "content": "What's the weather in Paris today?"}
    ],
    functions=[
        {
            "name": "get_current_weather",
            "description": "Get the current weather in a location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {"type": "string", "description": "City name"}
                },
                "required": ["location"]
            }
        }
    ],
    function_call="auto",
    max_tokens=50
)

In [11]:
# new Pydantic message object; function_call is an attribute 
message = response.choices[0].message
if message.function_call:  # :contentReference[oaicite:2]{index=2}
    args = json.loads(message.function_call.arguments)
    weather = get_current_weather(**args)
    print("Function call result:", weather)

Function call result: {'location': 'Paris', 'weather': 'sunny', 'temperature': '25°C'}
